<a href="https://colab.research.google.com/github/ranjini-rao/Neural-Network_PharmaProbe/blob/main/Clean_Neural_Net_No_PCA_for_Testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [64]:
import pandas as pd

# Load the CSV file into a DataFrame
embedded_df = pd.read_csv('embedded_review.csv')

# Display the first few rows of the DataFrame to verify it was loaded correctly
embedded_df.head()

,uniqueID,drugName,condition,review,rating,date,usefulCount,lengthReview,conditionCluster_label,drugNameCluster_label,...,758,759,760,761,762,763,764,765,766,767
0,95260,Guanfacine,ADHD,My son is halfway through his fourth week of I...,8.0,27-Apr-10,192.0,712.0,2.0,4.0,...,-0.416409,-0.364040,-0.036060,0.383963,0.176255,-0.147201,-0.243359,-0.541467,0.062160,0.049585
1,92703,Lybrel,Birth Control,I used to take another oral contraceptive whic...,5.0,14-Dec-09,17.0,708.0,9.0,6.0,...,-0.279307,-0.419729,-0.389261,0.328398,0.291834,-0.027217,-0.359790,-0.706709,0.047264,-0.017902
2,138000,Ortho Evra,Birth Control,This is my first time using any form of birth ...,8.0,3-Nov-15,10.0,428.0,9.0,4.0,...,-0.232733,-0.031823,-0.032784,0.188440,0.162272,0.363399,-0.096650,-0.693634,-0.024901,0.548486
3,35696,Buprenorphine naloxone,Opiate Dependence,Suboxone has completely turned my life around ...,9.0,27-Nov-16,37.0,669.0,0.0,2.0,...,-0.310564,-0.599643,-0.375174,0.309915,0.577983,0.051811,-0.184821,-0.710691,0.065533,0.371945
4,155963,Cialis,Benign Prostatic Hyperplasia,2nd day on 5mg started to work with rock hard ...,2.0,28-Nov-15,43.0,373.0,0.0,5.0,...,-0.247983,-0.438636,-0.037911,-0.030183,0.508780,0.064493,-0.205261,-0.527391,-0.101341,0.039573


In [65]:
len(embedded_df)

11316

In [66]:
#This Neural Net is just going to focus on classifying the Condition Cluster by the Review Embeddings
#This Neural Net is also going to use the dataframe that has not had dimensions reduced

# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf

In [67]:
# I am going to drop the columns that are not embeddings of the review and the condition_cluster

columns_to_drop = ['uniqueID', 'drugName', 'date', 'condition', 'review', 'lengthReview']

embedded_df.drop(columns=columns_to_drop, inplace=True)

In [68]:
#check the dataframe

embedded_df.head()

,rating,usefulCount,conditionCluster_label,drugNameCluster_label,0,1,2,3,4,5,...,758,759,760,761,762,763,764,765,766,767
0,8.0,192.0,2.0,4.0,-0.010977,0.010914,0.200967,-0.229490,-0.535286,0.012419,...,-0.416409,-0.364040,-0.036060,0.383963,0.176255,-0.147201,-0.243359,-0.541467,0.062160,0.049585
1,5.0,17.0,9.0,6.0,0.066320,0.189584,0.369006,-0.046920,-0.473988,-0.238288,...,-0.279307,-0.419729,-0.389261,0.328398,0.291834,-0.027217,-0.359790,-0.706709,0.047264,-0.017902
2,8.0,10.0,9.0,4.0,0.084101,-0.019134,0.294494,0.029783,-0.228783,0.170102,...,-0.232733,-0.031823,-0.032784,0.188440,0.162272,0.363399,-0.096650,-0.693634,-0.024901,0.548486
3,9.0,37.0,0.0,2.0,0.007820,0.207558,0.179105,-0.210057,-0.197015,0.104799,...,-0.310564,-0.599643,-0.375174,0.309915,0.577983,0.051811,-0.184821,-0.710691,0.065533,0.371945
4,2.0,43.0,0.0,5.0,-0.193177,0.360585,0.448292,-0.253824,-0.532782,0.085381,...,-0.247983,-0.438636,-0.037911,-0.030183,0.508780,0.064493,-0.205261,-0.527391,-0.101341,0.039573


In [69]:
#checking on why the values of the condition cluster look like a float

condition_value_counts = embedded_df['conditionCluster_label'].value_counts()
print(condition_value_counts)

0.0    4074
8.0    2100
2.0     955
1.0     865
7.0     777
9.0     752
4.0     736
5.0     356
3.0     353
6.0     347
Name: conditionCluster_label, dtype: int64


In [70]:
# When I start training the model I am getting an error messsage that NAN values are present
embedded_df = embedded_df.dropna(subset=['conditionCluster_label'])

In [71]:
# Split our preprocessed data into our features and target arrays
# Choosing randon_state = 30 to be able to re-create attempts later
# There is a lot of data, so I can train on 70%, setting test_size at .3
# This step is necessary for training the neural network to classify 10 categories
# This code is altered by chatgpt after I could not fix it on my own

from keras.utils import to_categorical

# Separate the target variable
target = embedded_df['conditionCluster_label']

# Verify the unique values in the target variable
print(target.unique())

# Extract features (excluding the target variable)
features = embedded_df.drop(columns=['conditionCluster_label'])

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.3, random_state=30)

# Convert target labels to one-hot encoded format
y_train_encoded = to_categorical(y_train, num_classes=10)
y_test_encoded = to_categorical(y_test, num_classes=10)

# Check the shapes of the one-hot encoded target labels
print("Shape of y_train_encoded:", y_train_encoded.shape)
print("Shape of y_test_encoded:", y_test_encoded.shape)

[2. 9. 0. 1. 3. 8. 5. 7. 4. 6.]
Shape of y_train_encoded: (7920, 10)
Shape of y_test_encoded: (3395, 10)


In [72]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [73]:
# Check the number of dimensions, make it a variable so it passes into

num_dimensions = embedded_df.shape[1]-1
print(num_dimensions)

771


In [74]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
# Most of this is going to be the same as the HW assignment, but there are 10 classifications to predict now
# So the output layer has been adjusted

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=771, activation="sigmoid", input_dim=num_dimensions))

# Second hidden layer
nn.add(Dense(units=600, activation='sigmoid'))

# Third hidden layer (example of adding an additional hidden layer)
nn.add(Dense(units=300, activation='sigmoid'))

#Fourth hidden layer (example of adding another hidden layer)
nn.add(Dense(units=150, activation='ELU'))

#Fifth hidden layer (example of adding another hidden layer)
nn.add(Dense(units=100, activation='leaky_relu'))

#Sixth hidden layer (example of adding another hidden layer)
nn.add(Dense(units=50, activation='relu'))

#Seventh hidden layer (example of adding another hidden layer)
nn.add(Dense(units=25, activation='ELU'))

#Eighth hidden layer (example of adding another hidden layer)
nn.add(Dense(units=15, activation='relu'))

# Output layer
nn.add(Dense(units=10, activation='softmax'))  # 10 units for 10 classes, softmax activation

# Check the structure of the model
nn.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_17 (Dense)            (None, 771)               595212    
                                                                 
 dense_18 (Dense)            (None, 600)               463200    
                                                                 
 dense_19 (Dense)            (None, 300)               180300    
                                                                 
 dense_20 (Dense)            (None, 150)               45150     
                                                                 
 dense_21 (Dense)            (None, 100)               15100     
                                                                 
 dense_22 (Dense)            (None, 50)                5050      
                                                                 
 dense_23 (Dense)            (None, 25)               

In [75]:
# Compile the model - the loss function is categorical and not for binary classification

nn.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

In [76]:
 # Fit the model to the training data
fit_model = nn.fit(X_train_scaled, y_train_encoded, epochs=10)

Epoch 1/10
248/248 [==============================] - 7s 21ms/step - loss: 1.8436 - accuracy: 0.3574
Epoch 2/10
248/248 [==============================] - 6s 23ms/step - loss: 1.5876 - accuracy: 0.3961
Epoch 3/10
248/248 [==============================] - 9s 37ms/step - loss: 1.4617 - accuracy: 0.4486
Epoch 4/10
248/248 [==============================] - 8s 34ms/step - loss: 1.3650 - accuracy: 0.4855
Epoch 5/10
248/248 [==============================] - 6s 25ms/step - loss: 1.2721 - accuracy: 0.5119
Epoch 6/10
248/248 [==============================] - 4s 18ms/step - loss: 1.1960 - accuracy: 0.5370
Epoch 7/10
248/248 [==============================] - 4s 18ms/step - loss: 1.1095 - accuracy: 0.5672
Epoch 8/10
248/248 [==============================] - 5s 20ms/step - loss: 1.0306 - accuracy: 0.6010
Epoch 9/10
248/248 [==============================] - 4s 17ms/step - loss: 0.9238 - accuracy: 0.6428
Epoch 10/10
248/248 [==============================] - 5s 20ms/step - loss: 0.8201 - accura

In [77]:
test_loss, test_accuracy = nn.evaluate(X_test_scaled, y_test_encoded)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

107/107 [==============================] - 1s 9ms/step - loss: 1.7478 - accuracy: 0.4489
Test Loss: 1.7478067874908447
Test Accuracy: 0.4488954246044159


In [80]:
#Get the test results from the above Neural Net

import numpy as np

# Get predicted probabilities for each class
y_pred_probabilities = nn.predict(X_test_scaled)

# Convert probabilities to predicted classes by selecting the class with the highest probability
y_pred = np.argmax(y_pred_probabilities, axis=1)

# Convert one-hot encoded y_test_encoded back to categorical labels
y_test_categorical = np.argmax(y_test_encoded, axis=1)

# Create a DataFrame to compare predicted and actual classes
result_df = pd.DataFrame({'Actual': y_test_categorical, 'Predicted': y_pred})

# Add a column indicating whether the prediction was correct
result_df['Correct'] = result_df['Actual'] == result_df['Predicted']

# Display the DataFrame
print(result_df)

# Calculate accuracy manually
accuracy = result_df['Correct'].mean()
print("Manual Test Accuracy:", accuracy)

107/107 [==============================] - 1s 12ms/step
      Actual  Predicted  Correct
0          9          9     True
1          0          4    False
2          0          8    False
3          8          0    False
4          8          7    False
...      ...        ...      ...
3390       9          9     True
3391       4          1    False
3392       4          0    False
3393       0          0     True
3394       1          0    False

[3395 rows x 3 columns]
Manual Test Accuracy: 0.4488954344624448


In [79]:
from google.colab import files

# Export result_df to a CSV file
result_df.to_csv('prediction_results.csv', index=False)

# Download the CSV file
files.download('prediction_results.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [81]:
#adjusting the learning rate

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import regularizers

# Define the model - deep neural net with L2 regularization
nn = Sequential()

# First hidden layer with L2 regularization
nn.add(Dense(units=771, activation="relu", kernel_regularizer=regularizers.l2(0.01), input_dim=num_dimensions))

# Second hidden layer with L2 regularization
nn.add(Dense(units=512, activation='relu', kernel_regularizer=regularizers.l2(0.01)))

# Third hidden layer with L2 regularization
nn.add(Dense(units=256, activation='relu', kernel_regularizer=regularizers.l2(0.01)))

# Fourth hidden layer with L2 regularization
nn.add(Dense(units=256, activation='relu', kernel_regularizer=regularizers.l2(0.01)))

# Fifth hidden layer with L2 regularization
nn.add(Dense(units=256, activation='relu', kernel_regularizer=regularizers.l2(0.01)))

# Sixth hidden layer with L2 regularization
nn.add(Dense(units=128, activation='relu', kernel_regularizer=regularizers.l2(0.01)))

# Seventh hidden layer with L2 regularization
nn.add(Dense(units=64, activation='relu', kernel_regularizer=regularizers.l2(0.01)))

# Eighth hidden layer with L2 regularization
nn.add(Dense(units=64, activation='relu', kernel_regularizer=regularizers.l2(0.01)))

# Output layer
nn.add(Dense(units=10, activation='softmax'))

# Define optimizer with custom learning rate
custom_optimizer = Adam(learning_rate=0.001)

# Compile the model with the custom optimizer
nn.compile(loss="categorical_crossentropy", optimizer=custom_optimizer, metrics=["accuracy"])

# Fit the model to the training data
fit_model = nn.fit(X_train_scaled, y_train_encoded, epochs=200)


Epoch 1/200
248/248 [==============================] - 9s 29ms/step - loss: 5.4045 - accuracy: 0.3569
Epoch 2/200
248/248 [==============================] - 7s 29ms/step - loss: 2.0712 - accuracy: 0.3600
Epoch 3/200
248/248 [==============================] - 6s 24ms/step - loss: 2.0065 - accuracy: 0.3614
Epoch 4/200
248/248 [==============================] - 6s 26ms/step - loss: 1.9707 - accuracy: 0.3640
Epoch 5/200
248/248 [==============================] - 6s 25ms/step - loss: 1.9298 - accuracy: 0.3727
Epoch 6/200
248/248 [==============================] - 5s 20ms/step - loss: 1.8996 - accuracy: 0.3732
Epoch 7/200
248/248 [==============================] - 5s 21ms/step - loss: 1.8787 - accuracy: 0.3706
Epoch 8/200
248/248 [==============================] - 7s 30ms/step - loss: 1.8573 - accuracy: 0.3708
Epoch 9/200
248/248 [==============================] - 4s 18ms/step - loss: 1.8436 - accuracy: 0.3756
Epoch 10/200
248/248 [==============================] - 5s 21ms/step - loss: 1.839

In [82]:
test_loss, test_accuracy = nn.evaluate(X_test_scaled, y_test_encoded)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

107/107 [==============================] - 1s 6ms/step - loss: 2.4738 - accuracy: 0.3770
Test Loss: 2.4738454818725586
Test Accuracy: 0.3770250380039215


In [83]:
#Get the test results from the above Neural Net

# Get predicted probabilities for each class
y_pred_probabilities = nn.predict(X_test_scaled)

# Convert probabilities to predicted classes by selecting the class with the highest probability
y_pred = np.argmax(y_pred_probabilities, axis=1)

# Convert one-hot encoded y_test_encoded back to categorical labels
y_test_categorical = np.argmax(y_test_encoded, axis=1)

# Create a DataFrame to compare predicted and actual classes
result_df = pd.DataFrame({'Actual': y_test_categorical, 'Predicted': y_pred})

# Add a column indicating whether the prediction was correct
result_df['Correct'] = result_df['Actual'] == result_df['Predicted']

# Display the DataFrame
print(result_df)

# Calculate accuracy manually
accuracy = result_df['Correct'].mean()
print("Manual Test Accuracy:", accuracy)

107/107 [==============================] - 1s 11ms/step
      Actual  Predicted  Correct
0          9          0    False
1          0          1    False
2          0          0     True
3          8          0    False
4          8          7    False
...      ...        ...      ...
3390       9          7    False
3391       4          1    False
3392       4          0    False
3393       0          0     True
3394       1          0    False

[3395 rows x 3 columns]
Manual Test Accuracy: 0.37702503681885124


In [84]:
from google.colab import files

# Export result_df to a CSV file
result_df.to_csv('prediction_results.csv', index=False)

# Download the CSV file
files.download('prediction_results.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [85]:
pip install keras-tuner

In [86]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from kerastuner.tuners import RandomSearch

# Define the model-building function
def build_model(hp):
    model = keras.Sequential()
    model.add(layers.Dense(units=hp.Int('units_input', min_value=64, max_value=800, step=64), activation='relu', input_dim=num_dimensions))
    for i in range(hp.Int('num_layers', 1, 8)):  # Number of hidden layers
        model.add(layers.Dense(units=hp.Int('units_' + str(i), min_value=64, max_value=512, step=64), activation='relu'))
    model.add(layers.Dense(10, activation='softmax'))
    model.compile(
        optimizer=keras.optimizers.Adam(
            hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])),
        loss='categorical_crossentropy',
        metrics=['accuracy'])
    return model

# Instantiate the tuner and perform hyperparameter search
tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=20,
    executions_per_trial=3,
    directory='my_dir',
    project_name='helloworld')

tuner.search(X_train_scaled, y_train_encoded, epochs=5, validation_split=0.2)

# Get the best hyperparameters
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"Optimal number of units in the input layer: {best_hps.get('units_input')}")
print(f"Optimal learning rate for the optimizer: {best_hps.get('learning_rate')}")
print(f"Optimal number of hidden layers: {best_hps.get('num_layers')}")
for i in range(best_hps.get('num_layers')):
    print(f"Optimal number of units in layer {i}: {best_hps.get('units_' + str(i))}")

# Build the model with the best hyperparameters and train it on the data
model = tuner.hypermodel.build(best_hps)
history = model.fit(X_train_scaled, y_train_encoded, epochs=50, validation_split=0.2)

# Evaluate the model on the test set
loss, accuracy = model.evaluate(X_test_scaled, y_test_encoded)
print("Test accuracy:", accuracy)

Trial 20 Complete [00h 01m 33s]
val_accuracy: 0.47453702489535016

Best val_accuracy So Far: 0.6660353541374207
Total elapsed time: 01h 06m 37s
Optimal number of units in the input layer: 192
Optimal learning rate for the optimizer: 0.0001
Optimal number of hidden layers: 5
Optimal number of units in layer 0: 256
Optimal number of units in layer 1: 64
Optimal number of units in layer 2: 64
Optimal number of units in layer 3: 64
Optimal number of units in layer 4: 64
Epoch 1/50
198/198 [==============================] - 4s 8ms/step - loss: 2.0044 - accuracy: 0.3382 - val_loss: 1.8381 - val_accuracy: 0.3763
Epoch 2/50
198/198 [==============================] - 1s 7ms/step - loss: 1.7532 - accuracy: 0.3742 - val_loss: 1.6833 - val_accuracy: 0.3908
Epoch 3/50
198/198 [==============================] - 1s 7ms/step - loss: 1.5417 - accuracy: 0.4433 - val_loss: 1.5671 - val_accuracy: 0.4198
Epoch 4/50
198/198 [==============================] - 1s 6ms/step - loss: 1.3627 - accuracy: 0.5073 - v

In [88]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import regularizers

# Define the model - deep neural net with L2 regularization and custom learning rate
nn = Sequential()

# First hidden layer with L2 regularization
nn.add(Dense(units=192, activation="relu", kernel_regularizer=regularizers.l2(0.01), input_dim=num_dimensions))

# Second hidden layer with L2 regularization
nn.add(Dense(units=256, activation='relu', kernel_regularizer=regularizers.l2(0.01)))

# Third hidden layer with L2 regularization
nn.add(Dense(units=64, activation='relu', kernel_regularizer=regularizers.l2(0.01)))

# Fourth hidden layer with L2 regularization
nn.add(Dense(units=64, activation='relu', kernel_regularizer=regularizers.l2(0.01)))

# Fifth hidden layer with L2 regularization
nn.add(Dense(units=64, activation='relu', kernel_regularizer=regularizers.l2(0.01)))

# Sixth hidden layer with L2 regularization
nn.add(Dense(units=64, activation='relu', kernel_regularizer=regularizers.l2(0.01)))

# Seventh hidden layer with L2 regularization
nn.add(Dense(units=10, activation='softmax'))

# Define optimizer with custom learning rate
custom_optimizer = Adam(learning_rate=0.0001)

# Compile the model with the custom optimizer
nn.compile(loss="categorical_crossentropy", optimizer=custom_optimizer, metrics=["accuracy"])

# Fit the model to the training data
fit_model = nn.fit(X_train_scaled, y_train_encoded, epochs=200)


Epoch 1/200
248/248 [==============================] - 4s 8ms/step - loss: 8.8904 - accuracy: 0.3299
Epoch 2/200
248/248 [==============================] - 2s 8ms/step - loss: 6.6617 - accuracy: 0.3717
Epoch 3/200
248/248 [==============================] - 2s 8ms/step - loss: 5.1718 - accuracy: 0.4207
Epoch 4/200
248/248 [==============================] - 2s 7ms/step - loss: 4.2070 - accuracy: 0.4604
Epoch 5/200
248/248 [==============================] - 1s 5ms/step - loss: 3.5699 - accuracy: 0.4847
Epoch 6/200
248/248 [==============================] - 1s 5ms/step - loss: 3.1328 - accuracy: 0.5008
Epoch 7/200
248/248 [==============================] - 1s 5ms/step - loss: 2.8216 - accuracy: 0.5210
Epoch 8/200
248/248 [==============================] - 2s 9ms/step - loss: 2.5881 - accuracy: 0.5461
Epoch 9/200
248/248 [==============================] - 2s 8ms/step - loss: 2.4105 - accuracy: 0.5590
Epoch 10/200
248/248 [==============================] - 2s 8ms/step - loss: 2.2689 - accura

In [89]:
test_loss, test_accuracy = nn.evaluate(X_test_scaled, y_test_encoded)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

107/107 [==============================] - 0s 2ms/step - loss: 3.0845 - accuracy: 0.4409
Test Loss: 3.0844857692718506
Test Accuracy: 0.44094255566596985


In [90]:
#Get the test results from the above Neural Net

# Get predicted probabilities for each class
y_pred_probabilities = nn.predict(X_test_scaled)

# Convert probabilities to predicted classes by selecting the class with the highest probability
y_pred = np.argmax(y_pred_probabilities, axis=1)

# Convert one-hot encoded y_test_encoded back to categorical labels
y_test_categorical = np.argmax(y_test_encoded, axis=1)

# Create a DataFrame to compare predicted and actual classes
result_df = pd.DataFrame({'Actual': y_test_categorical, 'Predicted': y_pred})

# Add a column indicating whether the prediction was correct
result_df['Correct'] = result_df['Actual'] == result_df['Predicted']

# Display the DataFrame
print(result_df)

# Calculate accuracy manually
accuracy = result_df['Correct'].mean()
print("Manual Test Accuracy:", accuracy)

107/107 [==============================] - 1s 5ms/step
      Actual  Predicted  Correct
0          9          0    False
1          0          0     True
2          0          8    False
3          8          6    False
4          8          7    False
...      ...        ...      ...
3390       9          2    False
3391       4          4     True
3392       4          4     True
3393       0          8    False
3394       1          2    False

[3395 rows x 3 columns]
Manual Test Accuracy: 0.4409425625920471


In [91]:
from google.colab import files

# Export result_df to a CSV file
result_df.to_csv('prediction_results.csv', index=False)

# Download the CSV file
files.download('prediction_results.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [92]:
#Lets add DrugName to the mix and see what we get

In [96]:
import pandas as pd

# Load the CSV file into a DataFrame
embedded_df = pd.read_csv('embedded_review.csv')

# Display the first few rows of the DataFrame to verify it was loaded correctly
embedded_df.head()

,uniqueID,drugName,condition,review,rating,date,usefulCount,lengthReview,conditionCluster_label,drugNameCluster_label,...,758,759,760,761,762,763,764,765,766,767
0,95260,Guanfacine,ADHD,My son is halfway through his fourth week of I...,8.0,27-Apr-10,192.0,712.0,2.0,4.0,...,-0.416409,-0.364040,-0.036060,0.383963,0.176255,-0.147201,-0.243359,-0.541467,0.062160,0.049585
1,92703,Lybrel,Birth Control,I used to take another oral contraceptive whic...,5.0,14-Dec-09,17.0,708.0,9.0,6.0,...,-0.279307,-0.419729,-0.389261,0.328398,0.291834,-0.027217,-0.359790,-0.706709,0.047264,-0.017902
2,138000,Ortho Evra,Birth Control,This is my first time using any form of birth ...,8.0,3-Nov-15,10.0,428.0,9.0,4.0,...,-0.232733,-0.031823,-0.032784,0.188440,0.162272,0.363399,-0.096650,-0.693634,-0.024901,0.548486
3,35696,Buprenorphine naloxone,Opiate Dependence,Suboxone has completely turned my life around ...,9.0,27-Nov-16,37.0,669.0,0.0,2.0,...,-0.310564,-0.599643,-0.375174,0.309915,0.577983,0.051811,-0.184821,-0.710691,0.065533,0.371945
4,155963,Cialis,Benign Prostatic Hyperplasia,2nd day on 5mg started to work with rock hard ...,2.0,28-Nov-15,43.0,373.0,0.0,5.0,...,-0.247983,-0.438636,-0.037911,-0.030183,0.508780,0.064493,-0.205261,-0.527391,-0.101341,0.039573


In [97]:
# I am going to drop the columns that are not embeddings of the review and the condition_cluster

columns_to_drop = ['uniqueID', 'drugNameCluster_label', 'date', 'condition', 'review', 'lengthReview']

embedded_df.drop(columns=columns_to_drop, inplace=True)

In [98]:
embedded_df.head()

,drugName,rating,usefulCount,conditionCluster_label,0,1,2,3,4,5,...,758,759,760,761,762,763,764,765,766,767
0,Guanfacine,8.0,192.0,2.0,-0.010977,0.010914,0.200967,-0.229490,-0.535286,0.012419,...,-0.416409,-0.364040,-0.036060,0.383963,0.176255,-0.147201,-0.243359,-0.541467,0.062160,0.049585
1,Lybrel,5.0,17.0,9.0,0.066320,0.189584,0.369006,-0.046920,-0.473988,-0.238288,...,-0.279307,-0.419729,-0.389261,0.328398,0.291834,-0.027217,-0.359790,-0.706709,0.047264,-0.017902
2,Ortho Evra,8.0,10.0,9.0,0.084101,-0.019134,0.294494,0.029783,-0.228783,0.170102,...,-0.232733,-0.031823,-0.032784,0.188440,0.162272,0.363399,-0.096650,-0.693634,-0.024901,0.548486
3,Buprenorphine naloxone,9.0,37.0,0.0,0.007820,0.207558,0.179105,-0.210057,-0.197015,0.104799,...,-0.310564,-0.599643,-0.375174,0.309915,0.577983,0.051811,-0.184821,-0.710691,0.065533,0.371945
4,Cialis,2.0,43.0,0.0,-0.193177,0.360585,0.448292,-0.253824,-0.532782,0.085381,...,-0.247983,-0.438636,-0.037911,-0.030183,0.508780,0.064493,-0.205261,-0.527391,-0.101341,0.039573


In [99]:
# Convert drugname into multiple columns of dummy variables

dummies_drugs = pd.get_dummies(embedded_df['drugName'])

# Concatenate dummy variables with the original DataFrame
embedded_df = pd.concat([
    embedded_df.drop(columns=['drugName']),
    dummies_drugs
], axis=1)

In [100]:
embedded_df.head()

,rating,usefulCount,conditionCluster_label,0,1,2,3,4,5,6,...,Zovirax Ointment,Zyban,Zyclara,Zymine,Zyprexa,Zyprexa Zydis,Zyrtec,Zyvox,ella,femhrt
0,8.0,192.0,2.0,-0.010977,0.010914,0.200967,-0.229490,-0.535286,0.012419,0.578620,...,0,0,0,0,0,0,0,0,0,0
1,5.0,17.0,9.0,0.066320,0.189584,0.369006,-0.046920,-0.473988,-0.238288,0.341089,...,0,0,0,0,0,0,0,0,0,0
2,8.0,10.0,9.0,0.084101,-0.019134,0.294494,0.029783,-0.228783,0.170102,0.185404,...,0,0,0,0,0,0,0,0,0,0
3,9.0,37.0,0.0,0.007820,0.207558,0.179105,-0.210057,-0.197015,0.104799,0.338058,...,0,0,0,0,0,0,0,0,0,0
4,2.0,43.0,0.0,-0.193177,0.360585,0.448292,-0.253824,-0.532782,0.085381,0.607802,...,0,0,0,0,0,0,0,0,0,0


In [101]:
# When I start training the model I am getting an error messsage that NAN values are present
embedded_df = embedded_df.dropna(subset=['conditionCluster_label'])

In [102]:
#data split step

from keras.utils import to_categorical

# Separate the target variable
target = embedded_df['conditionCluster_label']

# Verify the unique values in the target variable
print(target.unique())

# Extract features (excluding the target variable)
features = embedded_df.drop(columns=['conditionCluster_label'])

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.3, random_state=30)

# Convert target labels to one-hot encoded format
y_train_encoded = to_categorical(y_train, num_classes=10)
y_test_encoded = to_categorical(y_test, num_classes=10)

# Check the shapes of the one-hot encoded target labels
print("Shape of y_train_encoded:", y_train_encoded.shape)
print("Shape of y_test_encoded:", y_test_encoded.shape)

[2. 9. 0. 1. 3. 8. 5. 7. 4. 6.]
Shape of y_train_encoded: (7920, 10)
Shape of y_test_encoded: (3395, 10)


In [103]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [104]:
# Check the number of dimensions, make it a variable so it passes into

num_dimensions = embedded_df.shape[1]-1
print(num_dimensions)

2192


In [105]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
# Most of this is going to be the same as the HW assignment, but there are 10 classifications to predict now
# So the output layer has been adjusted

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=num_dimensions, activation="relu", input_dim=num_dimensions))

# Second hidden layer
nn.add(Dense(units=600, activation='leaky_relu'))

# Third hidden layer (example of adding an additional hidden layer)
nn.add(Dense(units=300, activation='relu'))

#Fourth hidden layer (example of adding another hidden layer)
nn.add(Dense(units=150, activation='ELU'))

#Fifth hidden layer (example of adding another hidden layer)
nn.add(Dense(units=100, activation='leaky_relu'))

#Sixth hidden layer (example of adding another hidden layer)
nn.add(Dense(units=50, activation='relu'))

#Seventh hidden layer (example of adding another hidden layer)
nn.add(Dense(units=25, activation='ELU'))

#Eighth hidden layer (example of adding another hidden layer)
nn.add(Dense(units=15, activation='relu'))

# Output layer
nn.add(Dense(units=10, activation='softmax'))  # 10 units for 10 classes, softmax activation

# Check the structure of the model
nn.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_23 (Dense)            (None, 2192)              4807056   
                                                                 
 dense_24 (Dense)            (None, 600)               1315800   
                                                                 
 dense_25 (Dense)            (None, 300)               180300    
                                                                 
 dense_26 (Dense)            (None, 150)               45150     
                                                                 
 dense_27 (Dense)            (None, 100)               15100     
                                                                 
 dense_28 (Dense)            (None, 50)                5050      
                                                                 
 dense_29 (Dense)            (None, 25)               

In [106]:
# Compile the model - the loss function is categorical and not for binary classification

nn.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

In [107]:
 # Fit the model to the training data
fit_model = nn.fit(X_train_scaled, y_train_encoded, epochs=10)

Epoch 1/10
248/248 [==============================] - 26s 96ms/step - loss: 1.5603 - accuracy: 0.4581
Epoch 2/10
248/248 [==============================] - 25s 99ms/step - loss: 0.8334 - accuracy: 0.7179
Epoch 3/10
248/248 [==============================] - 23s 91ms/step - loss: 0.5918 - accuracy: 0.7958
Epoch 4/10
248/248 [==============================] - 24s 99ms/step - loss: 0.4643 - accuracy: 0.8407
Epoch 5/10
248/248 [==============================] - 25s 100ms/step - loss: 0.3864 - accuracy: 0.8650
Epoch 6/10
248/248 [==============================] - 24s 95ms/step - loss: 0.3002 - accuracy: 0.8996
Epoch 7/10
248/248 [==============================] - 23s 91ms/step - loss: 0.2565 - accuracy: 0.9097
Epoch 8/10
248/248 [==============================] - 23s 93ms/step - loss: 0.2405 - accuracy: 0.9249
Epoch 9/10
248/248 [==============================] - 28s 113ms/step - loss: 0.1727 - accuracy: 0.9453
Epoch 10/10
248/248 [==============================] - 22s 91ms/step - loss: 0.1

In [108]:
test_loss, test_accuracy = nn.evaluate(X_test_scaled, y_test_encoded)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

107/107 [==============================] - 2s 16ms/step - loss: 1.3861 - accuracy: 0.7093
Test Loss: 1.3861266374588013
Test Accuracy: 0.7092783451080322


In [109]:
#Get the test results from the above Neural Net

# Get predicted probabilities for each class
y_pred_probabilities = nn.predict(X_test_scaled)

# Convert probabilities to predicted classes by selecting the class with the highest probability
y_pred = np.argmax(y_pred_probabilities, axis=1)

# Convert one-hot encoded y_test_encoded back to categorical labels
y_test_categorical = np.argmax(y_test_encoded, axis=1)

# Create a DataFrame to compare predicted and actual classes
result_df = pd.DataFrame({'Actual': y_test_categorical, 'Predicted': y_pred})

# Add a column indicating whether the prediction was correct
result_df['Correct'] = result_df['Actual'] == result_df['Predicted']

# Display the DataFrame
print(result_df)

# Calculate accuracy manually
accuracy = result_df['Correct'].mean()
print("Manual Test Accuracy:", accuracy)

107/107 [==============================] - 4s 38ms/step
      Actual  Predicted  Correct
0          9          9     True
1          0          0     True
2          0          0     True
3          8          4    False
4          8          0    False
...      ...        ...      ...
3390       9          9     True
3391       4          1    False
3392       4          4     True
3393       0          8    False
3394       1          4    False

[3395 rows x 3 columns]
Manual Test Accuracy: 0.709278350515464


In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from kerastuner.tuners import RandomSearch

# Define the model-building function
def build_model(hp):
    model = keras.Sequential()
    model.add(layers.Dense(units=hp.Int('units_input', min_value=64, max_value=800, step=64), activation='relu', input_dim=num_dimensions))
    for i in range(hp.Int('num_layers', 1, 8)):  # Number of hidden layers
        model.add(layers.Dense(units=hp.Int('units_' + str(i), min_value=64, max_value=512, step=64), activation='relu'))
    model.add(layers.Dense(10, activation='softmax'))
    model.compile(
        optimizer=keras.optimizers.Adam(
            hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])),
        loss='categorical_crossentropy',
        metrics=['accuracy'])
    return model

# Instantiate the tuner and perform hyperparameter search
tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=20,
    executions_per_trial=3,
    directory='my_dir',
    project_name='helloworld')

tuner.search(X_train_scaled, y_train_encoded, epochs=5, validation_split=0.2)

# Get the best hyperparameters
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"Optimal number of units in the input layer: {best_hps.get('units_input')}")
print(f"Optimal learning rate for the optimizer: {best_hps.get('learning_rate')}")
print(f"Optimal number of hidden layers: {best_hps.get('num_layers')}")
for i in range(best_hps.get('num_layers')):
    print(f"Optimal number of units in layer {i}: {best_hps.get('units_' + str(i))}")

# Build the model with the best hyperparameters and train it on the data
model = tuner.hypermodel.build(best_hps)
history = model.fit(X_train_scaled, y_train_encoded, epochs=50, validation_split=0.2)

# Evaluate the model on the test set
loss, accuracy = model.evaluate(X_test_scaled, y_test_encoded)
print("Test accuracy:", accuracy)

Reloading Tuner from my_dir/helloworld/tuner0.json

Search: Running Trial #2

Value             |Best Value So Far |Hyperparameter
576               |192               |units_input
6                 |5                 |num_layers
192               |256               |units_0
0.001             |0.0001            |learning_rate
384               |64                |units_1
128               |64                |units_2
192               |64                |units_3
384               |64                |units_4



Exception ignored in: <function _xla_gc_callback at 0x7fb69d02c040>
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/jax/_src/lib/__init__.py", line 97, in _xla_gc_callback
    def _xla_gc_callback(*args):
KeyboardInterrupt: 


Epoch 1/5
198/198 [==============================] - 6s 23ms/step - loss: 1.5118 - accuracy: 0.4508 - val_loss: 1.0989 - val_accuracy: 0.5903
Epoch 2/5
198/198 [==============================] - 6s 29ms/step - loss: 0.7239 - accuracy: 0.7334 - val_loss: 0.8871 - val_accuracy: 0.6963
Epoch 3/5
198/198 [==============================] - 4s 22ms/step - loss: 0.4490 - accuracy: 0.8406 - val_loss: 0.9307 - val_accuracy: 0.7058
Epoch 4/5
198/198 [==============================] - 5s 24ms/step - loss: 0.3406 - accuracy: 0.8755 - val_loss: 1.0384 - val_accuracy: 0.7165
Epoch 5/5
198/198 [==============================] - 5s 27ms/step - loss: 0.2573 - accuracy: 0.9132 - val_loss: 1.0797 - val_accuracy: 0.7045
Epoch 1/5
198/198 [==============================] - 7s 29ms/step - loss: 1.5103 - accuracy: 0.4680 - val_loss: 1.1325 - val_accuracy: 0.5922
Epoch 2/5
198/198 [==============================] - 4s 22ms/step - loss: 0.7407 - accuracy: 0.7320 - val_loss: 0.8725 - val_accuracy: 0.6963
Epoch 

KeyboardInterrupt: 

In [ ]:
#adjusting the learning rate

from tensorflow.keras.optimizers import Adam

# Define the model - deep neural net with L2 regularization
nn = Sequential()

# First hidden layer with L2 regularization
nn.add(Dense(units=771, activation="relu", kernel_regularizer=regularizers.l2(0.01), input_dim=num_dimensions))

# Second hidden layer with L2 regularization
nn.add(Dense(units=512, activation='relu', kernel_regularizer=regularizers.l2(0.01)))

# Third hidden layer with L2 regularization
nn.add(Dense(units=256, activation='relu', kernel_regularizer=regularizers.l2(0.01)))

# Fourth hidden layer with L2 regularization
nn.add(Dense(units=256, activation='relu', kernel_regularizer=regularizers.l2(0.01)))

# Fifth hidden layer with L2 regularization
nn.add(Dense(units=256, activation='relu', kernel_regularizer=regularizers.l2(0.01)))

# Sixth hidden layer with L2 regularization
nn.add(Dense(units=128, activation='relu', kernel_regularizer=regularizers.l2(0.01)))

# Seventh hidden layer with L2 regularization
nn.add(Dense(units=64, activation='relu', kernel_regularizer=regularizers.l2(0.01)))

# Eighth hidden layer with L2 regularization
nn.add(Dense(units=64, activation='relu', kernel_regularizer=regularizers.l2(0.01)))

# Output layer
nn.add(Dense(units=10, activation='softmax'))

# Define optimizer with custom learning rate
custom_optimizer = Adam(learning_rate=0.001)

# Compile the model with the custom optimizer
nn.compile(loss="categorical_crossentropy", optimizer=custom_optimizer, metrics=["accuracy"])

# Fit the model to the training data
fit_model = nn.fit(X_train_scaled, y_train_encoded, epochs=200)

Epoch 1/200
220/248 [=========================>....] - ETA: 1s - loss: 6.0349 - accuracy: 0.3598

KeyboardInterrupt: 

In [ ]:
test_loss, test_accuracy = nn.evaluate(X_test_scaled, y_test_encoded)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

107/107 [==============================] - 1s 8ms/step - loss: 2.3216 - accuracy: 0.3602
Test Loss: 2.3216376304626465
Test Accuracy: 0.36023563146591187


In [ ]:
import numpy as np

# Get predicted probabilities for each class
y_pred_probabilities = nn.predict(X_test_scaled)

# Convert probabilities to predicted classes by selecting the class with the highest probability
y_pred = np.argmax(y_pred_probabilities, axis=1)

# Convert one-hot encoded y_test_encoded back to categorical labels
y_test_categorical = np.argmax(y_test_encoded, axis=1)

# Create a DataFrame to compare predicted and actual classes
result_df = pd.DataFrame({'Actual': y_test_categorical, 'Predicted': y_pred})

# Add a column indicating whether the prediction was correct
result_df['Correct'] = result_df['Actual'] == result_df['Predicted']

# Display the DataFrame
print(result_df)

# Calculate accuracy manually
accuracy = result_df['Correct'].mean()
print("Manual Test Accuracy:", accuracy)


107/107 [==============================] - 1s 12ms/step
      Actual  Predicted  Correct
0          9          0    False
1          0          0     True
2          0          0     True
3          8          0    False
4          8          0    False
...      ...        ...      ...
3390       9          0    False
3391       4          0    False
3392       4          0    False
3393       0          0     True
3394       1          0    False

[3395 rows x 3 columns]
Manual Test Accuracy: 0.3602356406480118


In [110]:
from google.colab import files

# Export result_df to a CSV file
result_df.to_csv('prediction_results.csv', index=False)

# Download the CSV file
files.download('prediction_results.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>